In [18]:
import os
import pandas as pd
import numpy as np
import turicreate as tc
from sklearn.cross_validation import train_test_split

ModuleNotFoundError: No module named 'turicreate'

In [3]:
data = pd.read_csv('BlackFriday.csv')

In [4]:
print(data.shape)
data.head()

(537577, 12)


,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200
2,1000001,P00087842,F,0-17,10,A,2,0,12,NaN,NaN,1422
3,1000001,P00085442,F,0-17,10,A,2,0,12,14.0,NaN,1057
4,1000002,P00285442,M,55+,16,C,4+,0,8,NaN,NaN,7969


In [5]:
data_copy = data

In [6]:
data = data.drop(['Gender', 'Age', 'Occupation', 'City_Category', 'Stay_In_Current_City_Years', 'Marital_Status', 'Product_Category_1', 'Product_Category_2', 'Product_Category_3'], axis=1)


drop the column except for 'User_ID', 'Product_ID', 'Purchase'

In [12]:
# split dataset into training, test and validation set
def split_data(data, prop):
    train, test = train_test_split(data, test_size = prop)
    train_data = tc.SFrame(train)
    test_data = tc.SFrame(test)
    return train_data, test_data

In [16]:
data, data = split_data(data, 0.02)

NameError: name 'train_test_split' is not defined

In [19]:
 train_data, test_data = split_data(data, 0.2)

NameError: name 'train_test_split' is not defined

In [20]:
train_data, validate_data = split_data(train_data, 0.25)

NameError: name 'train_data' is not defined

In [23]:
#constant variables to define field names include :
user_id = 'User_ID'
item_id = 'Product_ID'
users_to_recommend = list(data[user_id])
n_rec = 5 # number of items to recommend
n_display = 30 # to display the first few rows in an output dataset

In [ ]:
def model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display):
    if name == 'popularity':
        model = tc.popularity_recommender.create(train_data,
                                                user_id = user_id,
                                                item_id = item_id,
                                                target = target)
    elif name == 'cosine':
        model = tc.item_similarity_recommender.create(train_data,
                                                     user_id = user_id,
                                                     item_id = item_id,
                                                     target = target,
                                                     similarity_type = 'cosine')
    elif name == 'pearson':
        model  = tc.item_similarity_recommender.create(train_data,
                                                      user_id = user_id,
                                                      item_id = item_id,
                                                      target = target,
                                                      similarity_type = 'pearson')
        
    recom = model.recommend(users= users_to_recommend, k=n_rec)
    recom.print_rows(n_display)
    return model

In [26]:
# Create two user_item matrices, one for training and another for testing
train_data_matrix = train_data.as_matrix(columns = ['user_id', 'item_id', 'target'])
test_data_matrix = test_data.as_matrix(columns = ['user_id', 'item_id', 'target'])
validate_data_matrix = validate_data.as_matrix(columns = ['user_id', 'item_id', 'target'])

#check their shape
print(train_data_matrix)
print(test_data_matrix)
print(validate_data_matrix)

NameError: name 'train_data' is not defined

In [27]:
from sklearn.metrices.pairwise import pairwise_distances

#User similarity Matrix
user_correlation = 1 - pairwise_distances(train_data, metric = 'correlation')
user_correlation[np.isnan(user_correlation)] = 0
print(user_correlation[:4, :4])


ModuleNotFoundError: No module named 'sklearn.metrices'

In [28]:
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_purchase = purchase.mean(axis = 1)
        #use np.newaxis so that mean_user_rating has same format as purchase
        purchase_diff = (ratings - mean_user_rating[:, np.newaxis])
        pred = mean_user_purchase[:, np.newaxis] + similarity.dot(purchase_diff) / np.array([np.abs(similarity).sum(axis = 1)]).T
    elif type == 'item':
        pred = purchase.dot(similarity) / np.array([np.abs(similarity).sum(axis = 1)])

## Evaluation


use RMSE

In [32]:
from sklearn.metrices import mean_squared_error
from math import sqrt

#Function to calculate RMSE
def rmse(pred,actual):
    #Ignore nonzero terms.
    pred = pred[actual.nonzero()].flatten()
    actual = pred[actual.nonzero()].flatten()
    return sqrt(mean_squared_error(pred,actual))

ModuleNotFoundError: No module named 'sklearn.metrices'

In [ ]:
user_prediction = predict(train_data_matrix, user_correlation, type=)